In [16]:
from bot import Bot
from game_config import Config, Blackjack
from shoe import Shoe

In [9]:
def simulate(n_player: int = 1, n_loop: int = 1000, faceup_target: str = None, hand_target: list = None):
    bet_amount = 1
    bot_signup = []
    for i in range(n_player):
        bot_signup.append(Bot(bankroll = 1_000_000, name = 'Player {}'.format(i+1)))

    config = Config() # Define your configuration game here. Check game_config.py for reference

    blackjack = Blackjack(config = config, player = bot_signup, shoe = Shoe(deck_quant = config.n_deck))
    card_quant = len(blackjack.shoe.shoe) # Total cards in the shoe

    loop = 0
    while loop < n_loop:
        game = blackjack.run()

        dealer_hand = game.dealer
        bots = game.players

        faceup_value = dealer_hand.get_faceup_card().card_face
        bots_value = [sorted([card.get_score()[0] for card in bot.hand], reverse=True) for bot in bots]

        if faceup_target:
            if faceup_target != faceup_value:
                continue
                
        if hand_target:
            if sorted(hand_target, reverse=True) not in bots_value:
                continue
        
        # Processing block
        for bot_hand in bots: # Player-scanning Block
            hand_process = [bot_hand]
            hand_count = 0

            while len(hand_process) != 0: # All-hands loop Block
                hand_in_play = hand_process[0]
                _count = 0

                quest = game.question(hand_in_play, _count)
                if hand_count != 0:
                    quest = 'Hand {} --- {}'.format(hand_count,quest)

				
				print('Bot Hand: {}'.format(hand_in_play.get_hand()))
				print('⍰ Quest: {}'.format(quest))
				while quest != None: # Response Block
					# Bot action
					action = hand_in_play.player.play(faceup_card = game.dealer.get_faceup_card(), hand = hand_in_play, question = quest)
					_count += 1
					print('--> {}'.format(action))

					if action == 'x2':
						hand_in_play.x2(blackjack.shoe)
						print(hand_in_play.get_hand())
					elif action == 'hit':
						hand_in_play.hit(blackjack.shoe)
						print(hand_in_play.get_hand())
					elif action == 'stand':
						hand_in_play.stand()
					elif action == 'split':
						hand_1, hand_2 = hand_in_play.split(blackjack.shoe)
						hand_process += [hand_1, hand_2]
						# hand_delete = True
						break

					quest = game.question(hand_in_play, _count)
					if quest != None:
						print('Quest: {}'.format(quest))

				hand_process.pop(0)

				hand_count += 1

		game.dealer_action(blackjack.shoe) 
		print('-----------------------------------------------')
		print(hand_print(dealer_hand, player=False))

		for bot_hand in bots: # Player-scanning Score block
			print('Bot {} ------ Results: '.format(bot_hand.player.name))
			hands = game.get_hands(player_name = bot_hand.player.name)
			for hand in hands:
				print(hand_print(hand))
				game.process_result(hand, bet_amount)

		blackjack.log(game)



        if len(blackjack.shoe.shoe) < int(card_quant*blackjack.config.reset_shoe):
            blackjack.shoe.shuffle(reset=True)


   